# Import Required Libraries

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd

import snowflake.connector as snow
from snowflake.connector.pandas_tools import write_pandas

import sqlalchemy as sql
import os

## Connection with Snowflake

In [2]:
conn = snow.connect(user="PB",
   password="Maruti@123",
   account="vw88853.ap-southeast-1",
   warehouse="DATALOADS",
   database="POC_DATASET",
   schema="POC_SCHEMA")


# Create a cursor object.
cur = conn.cursor()


## Load Data in Stock_fundamental Table

In [3]:
df = pd.read_excel('company.xlsx')
stock_id = 1
for i in df['Company']:
    SCRIP = i
    link = f'https://www.screener.in/company/{SCRIP}'
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = Request(link,headers=hdr)

    try:
        page=urlopen(req)
        soup = BeautifulSoup(page)

        div_html = soup.find('div',{'class': 'company-ratios'})
        ul_html = div_html.find('ul',{'id': 'top-ratios'})
        market_cap = 0.0
        dis = {}    
        for li in ul_html.find_all("li"):
            name_span = li.find('span',{'class':'name'})
            num_span = li.find('span',{'class':'nowrap value'})
            num_span = num_span.text.replace(',', '')
            num_span = num_span.replace(' ', '')
            num_span = num_span.replace('\n', '')
            num_span = num_span.replace('₹', '')
            num_span = num_span.replace('Cr.', '')
            num_span = num_span.replace('C', '')
            num_span = num_span.replace('%', '')
            dis[name_span.text.strip()] = num_span if (num_span != '') else 0.0

        dis['stock_name'] = soup.find('h1',{'class':'margin-0 show-from-tablet-landscape'}).text
        section_html = soup.find('section',{'id': 'peers'})
        div_html = section_html.find('div')
        sub_div = div_html.find('p',{'class':'sub'}).text.strip()
        sub_div = sub_div.replace('\n\n','@')
        sub_div = sub_div.replace(' ','')
        sub_div = sub_div.split('@')
        dis['sector'] = sub_div[0].replace('\n','').split(":")[1]
        dis['industry'] = sub_div[1].replace('\n','').split(":")[1]
        dis['symbol'] = SCRIP
#         cur.execute("INSERT INTO stock_fundamental(stock_id,stock_name,stock_symbol,pe,roce,book_value,roe,sector,industry,dividend_yield) VALUES("+str(stock_id)+",'"+dis['stock_name']+"','"+dis['symbol']+"',"+str(dis['Stock P/E'])+","+dis['ROCE']+","+dis['Book Value']+","+dis['ROE']+",'"+dis['sector']+"','"+dis['industry']+"',"+dis['Dividend Yield']+")")
        print(f'MARKET CAPITILIZATION - {SCRIP}: {dis}\n')
        stock_id += 1
    except:
        print(f'EXCEPTION THROWN: UNABLE TO FETCH DATA FOR {SCRIP}')
    

MARKET CAPITILIZATION - TCS: {'Market Cap': '1184069', 'Current Price': '3236', 'High / Low': '3739/2926', 'Stock P/E': '30.3', 'Book Value': '231', 'Dividend Yield': '1.35', 'ROCE': '61.3', 'ROE': '50.2', 'Face Value': '1.00', 'stock_name': 'Tata Consultancy Services Ltd', 'sector': 'IT-Software', 'industry': 'Computers-Software-Large', 'symbol': 'TCS'}

MARKET CAPITILIZATION - HCLTECH: {'Market Cap': '297146', 'Current Price': '1095', 'High / Low': '1165/876', 'Stock P/E': '25.6', 'Book Value': '149', 'Dividend Yield': '4.39', 'ROCE': '30.2', 'ROE': '25.0', 'Face Value': '2.00', 'stock_name': 'HCL Technologies Ltd', 'sector': 'IT-Software', 'industry': 'Computers-Software-Large', 'symbol': 'HCLTECH'}

MARKET CAPITILIZATION - WIPRO: {'Market Cap': '202479', 'Current Price': '369', 'High / Low': '584/355', 'Stock P/E': '16.3', 'Book Value': '107', 'Dividend Yield': '1.64', 'ROCE': '27.1', 'ROE': '24.4', 'Face Value': '2.00', 'stock_name': 'Wipro Ltd', 'sector': 'IT-Software', 'industry

In [ ]:
#cur.execute("INSERT INTO stock_fundamental(stock_id,stock_name,stock_symbol,pe,roce,book_value,roe,sector,industry,dividend_yield) VALUES("+str(stock_id)+",'"+dis['stock_name']+"','"+dis['symbol']+"',"+dis['Stock P/E']+","+dis['ROCE']+","+dis['Book Value']+","+dis['ROE']+",'"+dis['sector']+"','"+dis['industry']+"',"+dis['Dividend Yield']+")")

## Load Data in SHAREHOLDING_PATTERN Table

In [ ]:
cur.execute("SELECT stock_id,stock_name,stock_symbol FROM stock_fundamental")
one_row = cur.fetchall()
for i in one_row:
    shareholding = {'Promoters':0,'FIIs':0,'DIIs':0,'Government':0,'Public':0,'Others':0}
    stock = {}
    SCRIP = i[2]
    stock_name = i[1]
    stock_id =i[0]
    link = f'https://www.screener.in/company/{SCRIP}'
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = Request(link,headers=hdr)
    column = ""
    try:
        page=urlopen(req)
        soup = BeautifulSoup(page)

        section_html = soup.find('section',{'id': 'shareholding'})
        div_html = section_html.find('div',{'class': 'responsive-holder'})
        table_html = div_html.find('table',{'class': 'data-table'})
        thead_html = table_html.find('thead')
        trow_html = thead_html.find('tr')
        column = trow_html.text
        column = column.strip().split('\n\n')
        column = column[0].split('\n')
#         print(column)
        tbody_html = table_html.find('tbody')
        for trow_html in tbody_html.find_all("tr"):
            temp = []
            #trow_html = tbody_html.find('tr')
            ttitle_html = trow_html('td',{'class':'text'})

            title = trow_html.text
            title = title.strip().split('\n\n')
            header = title[0]
            header = header.replace('+','')
            header = header.replace('\n','')
#             print(header.strip()) 
            for i in range(1,len(title)):
                if title[i].split('\n') != "":
                    temp = title[i].split()
            stock[header.strip()] = temp
        print(stock_name)
        df =pd.DataFrame (stock, columns = list(stock.keys()),index = column)
        print(df)
        print('---------------------------------------------------------------------------')
        for j in range(0,len(df['Promoters'])):
            if 'Promoters' in df: shareholding['Promoters'] = df['Promoters'][j]
            if 'FIIs' in df: shareholding['FIIs'] = df['FIIs'][j]
            if 'DIIs' in df: shareholding['DIIs'] = df['DIIs'][j]
            if 'Government' in df: shareholding['Government'] = df['Government'][j]
            if 'Public' in df: shareholding['Public'] = df['Public'][j]
            if 'Others' in df: shareholding['Others'] = df['Others'][j]
            cur.execute("INSERT INTO SHAREHOLDING_PATTERN(stock_id,stock_name,stock_symbol,quarter_year,promotoer,fiis,diis,government,public,other) VALUES("+str(stock_id)+",'"+stock_name+"','"+SCRIP+"','"+column[j]+"','"+str(shareholding['Promoters'])+"','"+str(shareholding['FIIs'])+"','"+str(shareholding['DIIs'])+"','"+str(shareholding['Government'])+"','"+str(shareholding['Public'])+"','"+str(shareholding['Others'])+"')")

    except:
        print(f'EXCEPTION THROWN: UNABLE TO FETCH DATA FOR {SCRIP}')

    

## Load Data in Cash_flow Table

In [ ]:
cur.execute("SELECT stock_id,stock_name,stock_symbol FROM stock_fundamental")
one_row = cur.fetchall()

for i in one_row:
    cashflow = {'Cash from Operating Activity':0,'Cash from Investing Activity':0,'Cash from Financing Activity':0,'Net Cash Flow':0}
    stock = {}
    SCRIP = i[2]
    stock_name = i[1]
    stock_id =i[0]
    link = f'https://www.screener.in/company/{SCRIP}'
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = Request(link,headers=hdr)
    column = ""
    try:
        page=urlopen(req)
        soup = BeautifulSoup(page)

        section_html = soup.find('section',{'id': 'cash-flow'})
        div_html = section_html.find('div',{'class': 'responsive-holder'})
        table_html = div_html.find('table',{'class': 'data-table'})
        thead_html = table_html.find('thead')
        trow_html = thead_html.find('tr')
        column = trow_html.text
        column = column.strip().split('\n\n')
        for i in range(0,len(column)):
            column[i] = column[i].strip()

        tbody_html = table_html.find('tbody')
        for trow_html in tbody_html.find_all("tr"):
            temp = []
            #trow_html = tbody_html.find('tr')
            ttitle_html = trow_html('td',{'class':'text'})

            title = trow_html.text
            title = title.strip().split('\n\n')
            header = title[0]
            header = header.replace('+','')
            header = header.replace('\n','')
#             print(header.strip()) 
            for i in range(1,len(title)):
                if title[i] != "":
                    temp.append(title[i].strip())
            stock[header.strip()] = temp
#         print(stock)
        print('---------------------------------------------------------------------------')
        print(stock_name)
        df =pd.DataFrame (stock, columns = list(stock.keys()),index = column)
        print(df)
        
        for j in range(0,len(df['Cash from Operating Activity'])):
            if 'Cash from Operating Activity' in df: cashflow['Cash from Operating Activity'] = df['Cash from Operating Activity'][j]
            if 'Cash from Investing Activity' in df: cashflow['Cash from Investing Activity'] = df['Cash from Investing Activity'][j]
            if 'Cash from Financing Activity' in df: cashflow['Cash from Financing Activity'] = df['Cash from Financing Activity'][j]
            if 'Net Cash Flow' in df: cashflow['Net Cash Flow'] = df['Net Cash Flow'][j]
            cur.execute("INSERT INTO cash_flow(stock_id,stock_name,stock_symbol,year_month,cash_from_operating_activity,cash_from_investing_activity,cash_from_financing_activity,net_cash_flow) VALUES("+str(stock_id)+",'"+stock_name+"','"+SCRIP+"','"+column[j]+"','"+cashflow['Cash from Operating Activity']+"','"+cashflow['Cash from Investing Activity']+"','"+cashflow['Cash from Financing Activity']+"','"+cashflow['Net Cash Flow']+"')")

    except:
        print(f'EXCEPTION THROWN: UNABLE TO FETCH DATA FOR {SCRIP}')


In [ ]:
#cur.execute("INSERT INTO cash_flow(stock_id,stock_name,stock_symbol,year_month,cash_from_operating_activity,cash_from_investing_activity,cash_from_financing_activity,net_cash_flow) VALUES("+str(stock_id)+",'"+stock_name+"','"+SCRIP+"','"+column[j]+"',"+cashflow['Cash from Operating Activity']+","+cashflow['Cash from Investing Activity']+","+cashflow['Cash from Financing Activity']+","+cashflow['Net Cash Flow']+")")

## Load Data in Quarterly_result Table

In [ ]:
cur.execute("SELECT stock_id,stock_name,stock_symbol FROM stock_fundamental")
one_row = cur.fetchall()

for i in one_row:
    quarters = {'sales':0,'expenses':0,'operating_profit':0,'opm':0,'other':0,'interest':0,'depreciation':0,'profit':0,'tax':0,'net':0,'eps':0}
    stock = {}
    SCRIP = i[2]
    stock_name = i[1]
    stock_id =i[0]
    link = f'https://www.screener.in/company/{SCRIP}'
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = Request(link,headers=hdr)
    stock ={}
    try:
        page=urlopen(req)
        soup = BeautifulSoup(page)

        section_html = soup.find('section',{'id': 'quarters'})
        div_html = section_html.find('div',{'class': 'responsive-holder fill-card-width'})
        table_html = div_html.find('table',{'class': 'data-table'})
        thead_html = table_html.find('thead')
        trow_html = thead_html.find('tr')
        column = trow_html.text
        column = column.strip().split('\n\n')
        for i in range(0,len(column)):
            column[i] = column[i].strip()
        #print("hello " ,column)

        tbody_html = table_html.find('tbody')
        for trow_html in tbody_html.find_all("tr"):
            temp = []
            #trow_html = tbody_html.find('tr')
            ttitle_html = trow_html('td',{'class':'text'})
            title = trow_html.text
            title = title.replace('%','')
            title = title.strip().split('\n\n')
            header = title[0].split()[0]
            for i in range(1,len(title)):
                if title[i] != "":
                    temp.append(title[i].strip())
            stock[header] = temp

        #print(stock)
        del stock['Raw']
        df =pd.DataFrame (stock, columns = list(stock.keys()),index = column)
        print(df)
        for j in range(0,len(df['Sales'])):
            if 'Sales' in df: quarters['sales'] = df['Sales'][j]
            if 'Expenses' in df: quarters['expenses'] = df['Expenses'][j]
            if 'Operating' in df: quarters['operating_profit'] = df['Operating'][j]
            if 'OPM' in df: quarters['opm'] = df['OPM'][j]
            if 'Other' in df: quarters['other'] = df['Other'][j]
            if 'Interest' in df: quarters['interest'] = df['Interest'][j]
            if 'Depreciation' in df: quarters['depreciation'] = df['Depreciation'][j]
            if 'Profit' in df: quarters['profit'] = df['Profit'][j]
            if 'Tax' in df: quarters['tax'] = df['Tax'][j]
            if 'Net' in df: quarters['net'] = df['Net'][j]
            if 'Eps' in df: quarters['eps'] = df['Eps'][j]
            cur.execute("INSERT INTO quarterly_result(stock_id,stock_name,stock_symbol,quarter_year,sales,expenses,operating_profit,opm,other,interest,depreciation,profit,tax,net,eps) VALUES("+str(stock_id)+",'"+str(stock_name)+"','"+str(SCRIP)+"','"+str(column[j])+"','"+str(quarters['sales'])+"','"+str(quarters['expenses'])+"','"+str(quarters['operating_profit'])+"','"+str(quarters['opm'])+"','"+str(quarters['other'])+"','"+str(quarters['interest'])+"','"+str(quarters['depreciation'])+"','"+str(quarters['profit'])+"','"+str(quarters['tax'])+"','"+str(quarters['net'])+"','"+str(quarters['eps'])+"')")

    except:
        print(f'EXCEPTION THROWN: UNABLE TO FETCH DATA FOR {SCRIP}')

In [ ]:
#  cur.execute("INSERT INTO quarterly_result(stock_id,stock_name,stock_symbol,quarter_year,sales,expenses,operating_profit,opm,other,interest,depreciation,profit,tax,net,eps) VALUES("+str(stock_id)+",'"+stock_name+"','"+SCRIP+"','"+column[j]+"','"+quarters['sales'][j]+"','"+quarters['expenses'][j]+"','"+quarters['operating_profit'][j]+"','"+quarters['opm']+"','"+quarters['other']+"','"+quarters['interest']+"','"+quarters['depreciation']+"','"+quarters['profit']+"','"+quarters['tax']+"','"+quarters['net']+"',"+quarters['eps']+")")

## Load Data in Balance_sheet Table

In [ ]:
cur.execute("SELECT stock_id,stock_name,stock_symbol FROM stock_fundamental")
one_row = cur.fetchall()

for i in one_row:
    stock = {}
    SCRIP = i[2]
    stock_name = i[1]
    stock_id =i[0]
    link = f'https://www.screener.in/company/{SCRIP}'
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = Request(link,headers=hdr)
    balancesheet ={'share_capital':0,'reserves':0,'borrowings':0,'other_liabilities':0,'total_liabilities':0,'fixed_assets':0,'cwip':0,'investments':0,'other_assets':0,'total_assets':0}

    try:
        page=urlopen(req)
        soup = BeautifulSoup(page)

        section_html = soup.find('section',{'id': 'balance-sheet'})
        div_html = section_html.find('div',{'class': 'responsive-holder'})
        table_html = div_html.find('table',{'class': 'data-table'})
        thead_html = table_html.find('thead')
        trow_html = thead_html.find('tr')
        column = trow_html.text
        column = column.strip().split('\n\n')
        for i in range(0,len(column)):
            column[i] = column[i].strip()
        #print("hello " ,column)

        tbody_html = table_html.find('tbody')
        for trow_html in tbody_html.find_all("tr"):
            temp = []
            #trow_html = tbody_html.find('tr')
            ttitle_html = trow_html('td',{'class':'text'})

            title = trow_html.text
            title = title.strip().split('\n\n')
            header = title[0].strip()
            header = header.replace('+','')
            header = header.replace('\n','')
            header = header.replace('\xa0','')
            #print(header) 
            for i in range(1,len(title)):
                if title[i] != "":
                    temp.append(title[i].strip())
            stock[header] = temp
    #     print(stock)
        df =pd.DataFrame (stock, columns = list(stock.keys()),index = column)
        print(df)
        for j in range(0,len(df['Share Capital'])):
            if 'Share Capital' in df: balancesheet['share_capital'] = df['Share Capital'][j]
            if 'Borrowings' in df: balancesheet['borrowings'] = df['Borrowings'][j]
            if 'Reserves' in df: balancesheet['reserves'] = df['Reserves'][j]
            if 'Other Liabilities' in df: balancesheet['other_liabilities'] = df['Other Liabilities'][j]
            if 'Total Liabilities' in df: balancesheet['total_liabilities'] = df['Total Liabilities'][j]
            if 'Fixed Assets' in df: balancesheet['fixed_assets'] = df['Fixed Assets'][j]
            if 'CWIP' in df: balancesheet['cwip'] = df['CWIP'][j]
            if 'Investments' in df: balancesheet['investments'] = df['Investments'][j]
            if 'Other Assets' in df: balancesheet['other_assets'] = df['Other Assets'][j]
            if 'Total Assets' in df: balancesheet['total_assets'] = df['Total Assets'][j]
            cur.execute("INSERT INTO balance_sheet(stock_id,stock_name,stock_symbol,fin_month_year,share_capital,reserves,borrowings,other_liabilities,total_liabilities,fixed_assets,cwip,investments,other_assets,total_assets) VALUES("+str(stock_id)+",'"+str(stock_name)+"','"+str(SCRIP)+"','"+str(column[j])+"','"+str(balancesheet['share_capital'])+"','"+str(balancesheet['reserves'])+"','"+str(balancesheet['borrowings'])+"','"+str(balancesheet['other_liabilities'])+"','"+str(balancesheet['total_liabilities'])+"','"+str(balancesheet['fixed_assets'])+"','"+str(balancesheet['cwip'])+"','"+str(balancesheet['investments'])+"','"+str(balancesheet['other_assets'])+"','"+str(balancesheet['total_assets'])+"')")
           
    except:
        print(f'EXCEPTION THROWN: UNABLE TO FETCH DATA FOR {SCRIP}')

In [ ]:
cur.execute("INSERT INTO balance_sheet(stock_id,stock_name,stock_symbol,fin_month_year,share_capital,reserves,borrowings,other_liabilities,total_liabilities,fixed_assets,cwip,investments,other_assets,total_assets) VALUES("+str(stock_id)+",'"+str(stock_name)+"','"+str(SCRIP)+"','"+str(column[j])+"','"+str(balancesheet['share_capital'])+"','"+str(balancesheet['reserves'])+"','"+str(balancesheet['borrowings'])+"','"+str(balancesheet['other_liabilities'])+"','"+str(balancesheet['total_liabilities'])+"','"+str(balancesheet['fixed_assets'])+"','"+str(balancesheet['cwip'])+"','"+str(balancesheet['investments'])+"','"+str(balancesheet['other_assets'])+"','"+str(balancesheet['total_assets'])+"')")